In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        # Bidirectional LSTM layers
        self.bilstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout,
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_dim * 2, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc_out = nn.Linear(512, output_dim)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Embedding layer
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)

        # BiLSTM layers
        lstm_out, _ = self.bilstm(embedded)  # (batch_size, seq_len, hidden_dim * 2)

        # TimeDistributed fully connected layers
        output = self.fc1(lstm_out)  # (batch_size, seq_len, 512)
        output = F.relu(output)
        output = self.dropout(output)

        output = self.fc2(output)  # (batch_size, seq_len, 512)
        output = F.relu(output)
        output = self.dropout(output)

        output = self.fc_out(output)  # (batch_size, seq_len, output_dim)
        return F.log_softmax(output, dim=-1)


In [46]:
import sys
import pandas as pd
import json
sys.path.append("..")
from utils.data_preprocessing import preprocess_text
from utils.feature_extraction import bag_of_words, tfidf_features, extract_embeddings

train_path = '../dataset/PIZZA_train.json'
test_path = '../dataset/PIZZA_dev.json'
def read_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            try:
                record = json.loads(line.strip())
                data.append(record)
                
                # Process in chunks of 10,000 records
                if i > 0 and i % 10000 == 0:
                    print(f"Processed {i} records so far...")
            except json.JSONDecodeError:
                continue
    return data

# Convert remaining data to DataFrame
data = read_data(train_path)
if data:
    df = pd.DataFrame(data)
    
data = read_data(test_path)
if data:
    dev = pd.DataFrame(data)

Processed 10000 records so far...
Processed 20000 records so far...
Processed 30000 records so far...
Processed 40000 records so far...
Processed 50000 records so far...
Processed 60000 records so far...
Processed 70000 records so far...
Processed 80000 records so far...
Processed 90000 records so far...
Processed 100000 records so far...
Processed 110000 records so far...
Processed 120000 records so far...
Processed 130000 records so far...
Processed 140000 records so far...
Processed 150000 records so far...
Processed 160000 records so far...
Processed 170000 records so far...
Processed 180000 records so far...
Processed 190000 records so far...
Processed 200000 records so far...
Processed 210000 records so far...
Processed 220000 records so far...
Processed 230000 records so far...
Processed 240000 records so far...
Processed 250000 records so far...
Processed 260000 records so far...
Processed 270000 records so far...
Processed 280000 records so far...
Processed 290000 records so f

In [47]:
X_train = df['train.SRC']
y_train = df['train.EXR']
X_test = dev['dev.SRC']
y_test = dev['dev.EXR']
print(len(df))
print(X_train[2456445])
print(y_train[2456445])
print(dev['dev.SRC'][0])

2456446
i'd like a pizza with hot pepper pecorino cheese and parmesan without thin crust
(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING HOT_PEPPERS ) (TOPPING PECORINO_CHEESE ) (TOPPING PARMESAN_CHEESE ) (NOT (STYLE THIN_CRUST ) ) ) )
i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage


In [48]:
X_train = [" ".join(preprocess_text(text)) for text in X_train]
X_test = [" ".join(preprocess_text(text)) for text in X_test]
print(X_train[2456445])

d pizza hot pepper pecoricheese parmesan thin crust


In [49]:
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize_output(output):
    """
    Tokenizes the structured output into meaningful tokens.
    Example:
        Input: "(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )"
        Output: ["(ORDER", "(PIZZAORDER", "(NUMBER", "a", "(SIZE", "large", "(TOPPING", "bbq", "pulled", "pork", ")", ")", ")", ")"]
    """
    tokens = re.findall(r"\(|\)|\w+|[^\s()]+", output)
    return tokens

def build_vocab(outputs):
    """
    Builds a vocabulary from tokenized outputs.
    """
    vocab = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}  # Special tokens
    i = 2
    for output in outputs:
        tokens = tokenize_output(output)
        for token in tokens:
            if token not in vocab:
                vocab[token] = i
                i += 1
    return vocab
def encode_outputs(outputs, vocab):
    """
    Encodes tokenized outputs into sequences of integers.
    """
    encoded = []
    for output in outputs:
        tokens = tokenize_output(output)
        sequence = [vocab["<SOS>"]] + [vocab[token] for token in tokens if token in vocab] + [vocab["<EOS>"]]
        encoded.append(sequence)
    return encoded

def pad_sequences_to_fixed_length(sequences, max_len):
    """
    Pads sequences to a fixed length.
    """
    return pad_sequences(sequences, maxlen=max_len, padding="post", value=0)

def decode_sequence(sequence, vocab):
    """
    Decodes a sequence of integers back into the structured output string.
    """
    inv_vocab = {v: k for k, v in vocab.items()}  # Reverse the vocabulary
    tokens = [inv_vocab[idx] for idx in sequence if idx > 0]  # Ignore <PAD> tokens
    return " ".join(tokens)


In [68]:
import gensim.downloader as api
glove_vectors = api.load("glove-wiki-gigaword-100")  # 100-dimension GloVe
glove_vectors

In [69]:


def prepare_data(
    X_train, y_train, X_test, y_test, feature_type="bow", glove_vectors=None, max_len=20
):

    vectorizer = None

    # Feature Extraction for X_train and X_test
    if feature_type == "bow":
        X_train_processed, vectorizer = bag_of_words(X_train)
        X_train_processed = X_train_processed.toarray()
        X_test_processed = vectorizer.transform(X_test).toarray()
    elif feature_type == "tfidf":
        X_train_processed, vectorizer = tfidf_features(X_train)
        X_train_processed = X_train_processed.toarray()
        X_test_processed = vectorizer.transform(X_test).toarray()
    elif feature_type == "embeddings":
        if not glove_vectors:
            raise ValueError("GloVe vectors must be provided for embeddings.")
        X_train_tokenized = [sentence.split() for sentence in X_train]
        X_test_tokenized = [sentence.split() for sentence in X_test]
        X_train_processed = extract_embeddings(X_train_tokenized)
        X_test_processed = extract_embeddings(X_test_tokenized)
    else:
        raise ValueError("Invalid feature type. Choose 'bow', 'tfidf', or 'embeddings'.")

    vocab = build_vocab(y_train)  # Build vocabulary from training outputs
    y_train_encoded = encode_outputs(y_train, vocab)  # Encode training outputs
    y_test_encoded = encode_outputs(y_test, vocab)  # Encode testing outputs
    y_train_processed = pad_sequences_to_fixed_length(y_train_encoded, max_len)
    y_test_processed = pad_sequences_to_fixed_length(y_test_encoded, max_len)


    return (
        X_train_processed,
        X_test_processed,
        y_train_processed,
        y_test_processed,
        vectorizer,
        vocab,  # Return vocabulary for decoding
    )


In [70]:
X_train_processed, X_test_processed, y_train_processed, y_test_processed, vectorizer, vocab = prepare_data( X_train, y_train, X_test, y_test, feature_type="embeddings", max_len=300)

ValueError: GloVe vectors must be provided for embeddings.

In [61]:
X_train_processed.shape, y_train_processed.shape

((2456446, 797), (2456446, 300))

In [53]:
len(vocab)

183

In [54]:
input_dim = X_train_processed.shape[1]  # Vocabulary size
embedding_dim = 25  # Dimension of embedding vectors
hidden_dim = 256  # Hidden state size for LSTM
output_dim = y_train_processed.shape[1]  # Number of output classes
num_layers = 3  # Number of BiLSTM layers
dropout = 0.5  # Dropout probability

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.float32) 
        self.targets = torch.tensor(targets, dtype=torch.long) 

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "src_input_ids": self.inputs[idx],
            "tgt_input_ids": self.targets[idx]
        }

# if hasattr(X_train_processed, "toarray"):  # Check if it's a sparse matrix
#     X_train_processed = X_train_processed.toarray()

# if hasattr(y_train_processed, "toarray"):  # Same for targets
#     y_train_processed = y_train_processed.toarray()
    
train_dataset = SequenceDataset(X_train_processed, y_train_processed)
test_dataset = SequenceDataset(X_test_processed, y_test_processed)

batch_size = 128  # Adjust based GPU ;-;  memory
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 7831149848 bytes.

In [ ]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()  # Use for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):  # Number of epochs
    model.train()
    epoch_loss = 0
    total_batches = len(train_dataloader)
    progress_bar = tqdm(train_dataloader, desc="Training Progress", unit="batch", leave=True)
    for batch in train_dataloader:  # Assuming a DataLoader is used for batches
        src = batch["src_input_ids"].to(device)  # Input tokens
        tgt = batch["tgt_input_ids"].to(device)  # Target tokens

        optimizer.zero_grad()
        output = model(src)  # Forward pass

        # Reshape outputs and targets for loss computation
        output = output.view(-1, output_dim)
        tgt = tgt.view(-1)

        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_dataloader):.4f}")


In [ ]:
model.eval()
with torch.no_grad():
    sample_input = torch.tensor([[1, 2, 3, 4, 5]], dtype=torch.long).to(device)  # Example input
    output = model(sample_input)
    predicted_classes = torch.argmax(output, dim=-1)  # Get class with highest probability
    print(predicted_classes)
